please don't beat me please don't beat me let me live

In [1]:
import random
import numpy as np
import pickle

from fireplace import cards
from fireplace.card import Card
from fireplace.exceptions import GameOver, InvalidAction
from fireplace.game import Game
from fireplace.player import Player
from fireplace.utils import random_draft
from hearthstone.enums import CardClass, CardType, State
from fireplace.exceptions import GameOver

In [2]:
import torch
import torch.nn as nn
import torch.backends.cudnn as cudnn
import torch.optim as optim
import torch.utils.data
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torchvision.utils as vutils
import numpy as np
import matplotlib.pyplot as plt

In [3]:
cards.db.initialize()

[fireplace.__init__]: Initializing card database
[fireplace.__init__]: Merged 13764 cards


In [4]:
n_in = 100
n_out = 21*16

num_epochs = 50
batch_size = 128
lr = 1e-4

ngpu = 1
device = torch.device("cuda:0" if (torch.cuda.is_available() and ngpu > 0) else "cpu")

In [5]:
class NN(nn.Module):
    def __init__(self, ngpu):
        super(NN, self).__init__()
        self.ngpu = ngpu
        self.dense1 = nn.Linear(n_in, 64, bias=False)
        self.dense2 = nn.Linear(64, 32, bias=False)
        self.dense3 = nn.Linear(32, n_out, bias=False)

    def forward(self, input):
        x = F.relu(self.dense1(x))
        x = F.relu(self.dense2(x))
        x = F.Tanh(self.dense3(x))
        return x   

In [6]:
net = NN(ngpu).to(device)

if (device.type == 'cuda') and (ngpu > 1):
    net = nn.DataParallel(netG, list(range(ngpu)))

print(net)

NN(
  (dense1): Linear(in_features=100, out_features=64, bias=False)
  (dense2): Linear(in_features=64, out_features=32, bias=False)
  (dense3): Linear(in_features=32, out_features=336, bias=False)
)


In [7]:
criterion = nn.MSELoss()
optimizer = optim.Adam(net.parameters(), lr=lr)

In [8]:
def InitGame():
    c1 = CardClass(random.randint(2, 10))
    c2 = CardClass(random.randint(2, 10))
    deck1 = random_draft(c1)
    deck2 = random_draft(c2)
    
    players = []
    players.append(Player("Player1", deck1, c1.default_hero))
    players.append(Player("Player2", deck2, c2.default_hero))
    game = Game(players = players)
    game.start()

    return game

In [9]:
def MulliganChoice(cards):
    return random.sample(cards, random.randint(0, 3))

In [10]:
newgame = InitGame()

for player in newgame.players:
    cards_to_mulligan = MulliganChoice(player.choice.cards)
    player.choice.choose(*cards_to_mulligan)

[fireplace.entity]: Setting up game Game(players=[Player(name='Player1', hero=None), Player(name='Player2', hero=None)])
[fireplace.entity]: Tossing the coin... Player1 wins!
[fireplace.actions]: Player(name='Player1', hero=<Hero ('Anduin Wrynn')>) triggering <TargetedAction: Summon(<Summon.CARD>=<HeroPower ('Lesser Heal')>)> targeting [Player(name='Player1', hero=<Hero ('Anduin Wrynn')>)]
[fireplace.actions]: Player1 summons [<HeroPower ('Lesser Heal')>]
[fireplace.entity]: Empty stack, refreshing auras and processing deaths
[fireplace.actions]: Player(name='Player1', hero=<Hero ('Anduin Wrynn')>) triggering <TargetedAction: Summon(<Summon.CARD>=<Hero ('Anduin Wrynn')>)> targeting [Player(name='Player1', hero=<Hero ('Anduin Wrynn')>)]
[fireplace.actions]: Player1 summons [<Hero ('Anduin Wrynn')>]
[fireplace.entity]: Empty stack, refreshing auras and processing deaths
[fireplace.entity]: Player(name='Player1', hero=<Hero ('Anduin Wrynn')>) shuffles their deck
[fireplace.card]: <Minion 

In [13]:
moves = getValidMoves(newgame)
moves

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,

In [14]:
for player in newgame.players:
    print(player, ":", player.hand)

Player1 : [<Minion ('Frozen Crusher')>, <Minion ('Ancient Watcher')>, <Minion ('Barrens Blacksmith')>, <Minion ('Reckless Experimenter')>]
Player2 : [<Minion ('Eydis Darkbane')>, <Minion ('The Glass Knight')>, <Minion ('Proud Defender')>, <Minion ('Spiritsinger Umbra')>, <Spell ('The Coin')>]


In [15]:
print(newgame.current_player)
print(newgame.state)
print(newgame.current_player.mana)
for card in newgame.current_player.hand:
    print([card.is_playable(),card.requires_target(),card.data.cost])

print(newgame.players[0].hero.health)
print(newgame.players[1].hero.health)

newgame.current_player.field

Player1
State.RUNNING
1
[False, False, 6]
[False, False, 2]
[False, False, 5]
[False, False, 5]
30
30


[]

In [ ]:
a = [19,0]
getAction(a, newgame)

In [16]:
newgame.ended

False

In [25]:
def getNextState(player, action, game):
    try:
        DoAction(action, game)
    except GameOver:
        raise GameOver

    next_state = getState(game)
    
    return next_state, player

def getValidMoves(game):
    actions = np.zeros((21,16))
    player = game.current_player
    if player.choice:
        for index, card in enumerate(player.choice.cards):
            actions[20, index] = 1

    else:
        for index, card in enumerate(player.hand):
            if card.is_playable():
                if card.requires_target():
                    for target, card in enumerate(card.targets):
                        actions[index, target] = 1
                else:
                    actions[index] = 1
        for position, minion in enumerate(player.field):
            if minion.can_attack():
                for target, card in enumerate(minion.attack_targets):
                    actions[position+10, target] = 1
        if player.hero.power.is_usable():
            if player.hero.power.requires_target():
                for target, card in enumerate(player.hero.power.targets):
                    actions[17, target] = 1
            else:
                actions[17] = 1
        if player.hero.can_attack():
            for target, card in enumerate(player.hero.attack_targets):
                actions[18, target] = 1
        actions[19,0] = 1
    return actions

def getAction(a, game):
    player = game.current_player
    if not game.ended:
        try:
            if 0 <= a[0] <= 9:
                if player.hand[a[0]].requires_target():
                    player.hand[a[0]].play(player.hand[a[0]].targets[a[1]])
                else:
                    player.hand[a[0]].play()
            elif 10 <= a[0] <= 16:
                player.field[a[0]-10].attack(player.field[a[0]-10].attack_targets[a[1]])
            elif a[0] == 17:
                if player.hero.power.requires_target():
                        player.hero.power.use(player.hero.power.play_targets[a[1]])
                else:
                    player.hero.power.use()
            elif a[0] == 18:
                player.hero.attack(player.hero.attack_targets[a[1]])
            elif a[0] == 19:
                player.game.end_turn()
            elif a[0] == 20 and not player.choice:
                player.game.end_turn()
            elif player.choice:
                player.choice.choose(player.choice.cards[a[1]])
            else:
                print("Not an appropriate action")
        except InvalidAction:
            print("InvalidAction")
            player.game.end_turn()
        except IndexError:
            try:
                player.game.end_turn()
            except GameOver:
                pass
        except GameOver:
            pass

#calculating reward
def EndState(game):
    p1 = game.players[0]

    if p1.playstate == 4:
        return 1
    elif p1.playstate == 5:
        return -1
    elif p1.playstate == 6:
        return 1e-4
    elif game.turn > 180:
        game.ended = True
        return 1e-4
    return 0

#getting state for network
def getState(game):
    s = np.zeros((34,16), dtype = 'int32')

    p1 = game.current_player
    p2 = p1.opponent

    s[0] = p1.hero.health
    s[1] = p2.hero.health
    s[2] = p1.max_mana
    s[3] = p2.max_mana
    s[4] = p1.mana
    
    #hero player1
    i = 5
    s[i,0] = p1.hero.card_class
    s[i,1] = p1.hero.power.is_usable()*1
    s[i,2] = p1.hero.power.cost             #add effect
    if p1.weapon is None:
        s[i,2:5] = 0
    else:
        s[i,3] = 1
        s[i,4] = p1.weapon.damage
        s[i,5] = p1.weapon.durability     #add effect, attack, deathrattle
        
    #hero player2
    i = 6
    s[i,0] = p2.hero.card_class
    s[i,1] = p2.hero.power.is_usable()*1
    s[i,2] = p2.hero.power.cost            #add effect
    if p2.weapon is None:
        s[i,2:5] = 0
    else:
        s[i,3] = 1
        s[i,4] = p2.weapon.damage
        s[i,5] = p2.weapon.durability  #add effect, attack, deathrattle
    
    i = 7
    hand = len(p1.hand)
    s[i] = hand
    s[i + 1] = len(p2.hand)
    
    #hand
    i = 9
    for j in range(0, 10):
        if j < hand:
            s[i + j, 0] = 1
            s[i + j, 1] = p1.hand[j].cost
            s[i + j, 2] = p1.hand[j].card_class
            s[i + j, 3] = p1.hand[j].type
            if (p1.hand[j].type == 4):
                s[i + j, 4] = p1.hand[j].race 
                s[i + j, 5] = p1.hand[j].atk
                s[i + j, 6] = p1.hand[j].max_health
                s[i + j, 7] = p1.hand[j].divine_shield*1 
                s[i + j, 8] = p1.hand[j].has_deathrattle*1 
                s[i + j, 9] = p1.hand[j].has_battlecry*1
                s[i + j, 10] = p1.hand[j].taunt*1 
        
    #field
    i = 19
    f1 = len(p1.field)
    for j in range(0, 7):
        if j < f1:
            s[i + j, 0] = 1
            s[i + j, 1] = p1.field[j].health
            s[i + j, 2] = p1.field[j].card_class
            s[i + j, 3] = p1.field[j].can_attack()*1
            
            s[i + j, 4] = p1.field[j].race 
            s[i + j, 5] = p1.field[j].atk
            s[i + j, 6] = p1.field[j].max_health
            s[i + j, 7] = p1.field[j].divine_shield*1 
            s[i + j, 8] = p1.field[j].has_deathrattle*1
            s[i + j, 9] = p1.field[j].has_battlecry*1
            s[i + j, 10] = p1.field[j].taunt*1 
            s[i + j, 11] = p1.field[j].stealthed*1
            s[i + j, 12] = p1.field[j].silenced*1

    i = 26        
    f2 = len(p2.field)
    for j in range(0, 7):
        if j < f2:
            s[i + j, 0] = 1
            s[i + j, 1] = p2.field[j].health
            s[i + j, 2] = p2.field[j].card_class
            s[i + j, 3] = p2.field[j].can_attack()*1
            
            s[i + j, 4] = p2.field[j].race 
            s[i + j, 5] = p2.field[j].atk
            s[i + j, 6] = p2.field[j].max_health
            s[i + j, 7] = p2.field[j].divine_shield*1 
            s[i + j, 8] = p2.field[j].has_deathrattle*1
            s[i + j, 9] = p2.field[j].has_battlecry*1
            s[i + j, 10] = p2.field[j].taunt*1 
            s[i + j, 11] = p2.field[j].stealthed*1
            s[i + j, 12] = p2.field[j].silenced*1
        i += 1
        
    return s

In [26]:
s = getState(newgame)

In [27]:
s

array([[30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30],
       [30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30],
       [ 1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1],
       [ 6,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 5,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4],
       [ 5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5],
       [ 1,  6, 12,  4, 18,  8,  8,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 1,  2, 12,  4,  0,  4,  5,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 1,  5, 12,  4,  0,  3,  5,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 1,  5,  6,  4,  0,  4,  6,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,

In [54]:
card = Card("BAR_026")

card.max_health

4

In [32]:
dir(card.data)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'adapt',
 'appear_functionally_dead',
 'armor',
 'artist',
 'atk',
 'autoattack',
 'avenge',
 'battlecry',
 'can_summon_maxplusone_minion',
 'cant_be_attacked',
 'cant_be_fatigued',
 'card_class',
 'card_id',
 'card_set',
 'choose_cards',
 'classes',
 'collectible',
 'corrupt',
 'cost',
 'craftable',
 'crafting_costs',
 'dbf_id',
 'deathrattle',
 'description',
 'discover',
 'disenchant_costs',
 'divine_shield',
 'dormant',
 'double_spelldamage_bonus',
 'durability',
 'echo',
 'elite',
 'entourage',
 'evil_glow',
 'faction',
 'flavortext',
 'forgetful',
 'from_xml',
 'ghostly',
 'grimy_goons',
 'health',
 'hero_power',
 '

In [29]:
minions = cards.filter(
    collectible=True,
    cost=3,
    type=CardType.MINION
)
new = []
for x in minions:
    if Card(x).taunt == True:
        new.append(x)
new

['BAR_026',
 'BOT_021',
 'BT_136',
 'BT_730',
 'CFM_653',
 'CFM_815',
 'CS2_125',
 'CS2_127',
 'DMF_081',
 'DMF_184',
 'DMF_247',
 'EX1_301',
 'EX1_390',
 'FP1_009',
 'GIL_207',
 'GVG_097',
 'GVG_098',
 'ICC_081',
 'ICC_808',
 'LOE_022',
 'OG_327',
 'TRL_551',
 'ULD_250',
 'UNG_072',
 'UNG_928',
 'UNG_938',
 'VAN_CS2_125',
 'VAN_CS2_127',
 'VAN_EX1_301',
 'VAN_EX1_390',
 'WC_004',
 'YOP_025']